In [3]:
import numpy as np
from sklearn.linear_model import LinearRegression

# 1. Generate some sample data
X = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]).reshape(-1, 1) # Features
y = np.array([2, 4, 5, 4, 5, 7, 8, 9, 10, 12]) # Target variable

# 2. Define the model
model = LinearRegression()

# 3. Train the model
model.fit(X, y)

# 4. Make a prediction
new_X = np.array([11]).reshape(-1, 1)
prediction = model.predict(new_X)

print(f"Trained model coefficient: {model.coef_[0]:.2f}")
print(f"Trained model intercept: {model.intercept_:.2f}")
print(f"Prediction for new data {new_X[0][0]}: {prediction[0]:.2f}")

ModuleNotFoundError: No module named 'sklearn'

In [4]:
pip install scikit-learn

^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from transformers import pipeline

pipe = pipeline("text-classification")

In [ ]:
data = pipe("This restaurant is not awesome")
data[0]['label']


https://huggingface.co/docs/transformers/v4.57.0/en/main_classes/pipelines#transformers.Pipeline.model



In [ ]:
from transformers import pipeline

pipeline = pipeline(task="text-generation", model="google/gemma-2-2b")
pipeline("the secret to baking a really good cake is ")
[{'generated_text': 'the secret to baking a really good cake is 1. the right ingredients 2. the'}]

In [ ]:
import datasets
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm

pipe = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-base-960h", device=0)
dataset = datasets.load_dataset("superb", name="asr", split="test")

# KeyDataset (only *pt*) will simply return the item in the dict returned by the dataset item
# as we're not interested in the *target* part of the dataset. For sentence pair use KeyPairDataset
for out in tqdm(pipe(KeyDataset(dataset, "file"))):
    print(out)
    # {"text": "NUMBER TEN FRESH NELLY IS WAITING ON YOU GOOD NIGHT HUSBAND"}
    # {"text": ....}
    # ....

    # hf_ CXeDvBPImWhKBaiJ
    # BFfuuKpqlQKxjIgIIA


In [2]:
pip install transformers torch pillow requests


  Using cached pillow-12.0.0-cp310-cp310-win_amd64.whl.metadata (9.0 kB)
  Using cached numpy-2.2.6-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached pyyaml-6.0.3-cp310-cp310-win_amd64.whl.metadata (2.4 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached markupsafe-3.0.3-cp310-cp310-win_amd64.whl.metadata (2.8 kB)
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ---- ----------------------------------- 1.3/12.0 MB 8.4 MB/s eta 0:00:02
   ------------ --------------------------- 3.7/12.0 MB 9.9 MB/s eta 0:00:01
   ------------------- -------------------- 5.8/12.0 MB 10.4 MB/s eta 0:00:01
   ------------------------ --------------- 7.3/12.0 MB 9.3 MB/s eta 0:00:01
   -------------

In [1]:
3

3

Alright, let’s build a **free image generation setup using Hugging Face + Transformers** 😎

I’ll show you two options:

1. **Completely free & easiest** → Use **Hugging Face Inference API** (no GPU needed, runs on HF servers)
2. **Local free (needs GPU/strong CPU)** → Download model and run via `transformers` / `diffusers`

Since you wrote *“using tranfrmer”*, I’ll keep it focused on Hugging Face ecosystem (Transformers-style pipelines).

---

## 1️⃣ Setup – Hugging Face Account & Token (One Time)

1. Go to Hugging Face → create a free account.
2. Go to **Settings → Access Tokens** → create a **Read token**.
3. Install required libraries:

```bash
pip install transformers torch pillow requests
```

If you want to use diffusion models (recommended for images):

```bash
pip install diffusers accelerate safetensors
```

---

## 2️⃣ Easiest Way: Use Hugging Face Inference API (Free Tier)

Here we **don’t download** the model; we just call it using HTTPS.
Example using a Stable Diffusion model (text → image):

```python
import requests

API_URL = "https://api-inference.huggingface.co/models/stabilityai/stable-diffusion-2-1"
HF_TOKEN = "YOUR_HF_API_TOKEN"   # <--- paste your token here

headers = {"Authorization": f"Bearer HF_TOKEN_REPLACE"}

def generate_image(prompt, output_path="generated.png"):
    payload = {"inputs": prompt}
    response = requests.post(API_URL, headers=headers, json=payload)

    if response.status_code != 200:
        print("Error:", response.status_code, response.text)
        return

    with open(output_path, "wb") as f:
        f.write(response.content)
    print(f"Saved image to {output_path}")


if __name__ == "__main__":
    prompt = "a futuristic city at sunset, ultra realistic, 4k"
    generate_image(prompt)
```

👉 **Important:** Replace

```python
headers = {"Authorization": f"Bearer HF_TOKEN_REPLACE"}
```

with

```python
headers = {"Authorization": f"Bearer " + HF_TOKEN}
```

This gives you a **free image generator** using a Hugging Face model, no GPU on your side.

---

## 3️⃣ Using `diffusers` + Transformers Locally (Open-Source, Free)

This uses a **Transformers-style pipeline** but via the `diffusers` library (Hugging Face official for diffusion models).

```python
from diffusers import StableDiffusionPipeline
import torch

# Download any free text-to-image model, example:
model_id = "runwayml/stable-diffusion-v1-5"

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,  # use float32 if you don't have GPU
)

# If you have a GPU (recommended)
if torch.cuda.is_available():
    pipe = pipe.to("cuda")

prompt = "a cute robot teaching data science, digital art, vibrant colors"
image = pipe(prompt).images[0]

image.save("robot_ds.png")
print("Image saved as robot_ds.png")
```

If you **don’t** have GPU, change to:

```python
pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float32
).to("cpu")
```

⚠️ CPU will be **very slow** but still free.

---

## 4️⃣ “Transformers” Style Pipeline (Non-diffusers models)

Some image models are available directly via `transformers` pipelines (e.g., text-to-image, image-to-image). Example:

```python
from transformers import AutoProcessor, AutoModelForCausalLM
from PIL import Image
import torch

# Example: Phi-3-Vision / Qwen-VL etc. can generate images or reason about them
# (Many models are more multimodal than pure generators)
```

But for **pure image generation**, **diffusers** is the normal + recommended way in HF ecosystem, and it’s built to work **with transformers under the hood**.

---

## 5️⃣ Wrap It as a Simple “Free HF Image Generator” Function

Here’s a neat utility that automatically uses GPU if available:

```python
from diffusers import StableDiffusionPipeline
import torch

class HFImageGenerator:
    def __init__(self, model_id="runwayml/stable-diffusion-v1-5"):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        dtype = torch.float16 if self.device == "cuda" else torch.float32

        self.pipe = StableDiffusionPipeline.from_pretrained(
            model_id,
            torch_dtype=dtype,
        ).to(self.device)

    def generate(self, prompt: str, file_name: str = "output.png"):
        image = self.pipe(prompt).images[0]
        image.save(file_name)
        print(f"✅ Saved: {file_name}")

if __name__ == "__main__":
    gen = HFImageGenerator()
    gen.generate("a fantasy landscape with floating islands, matte painting", "fantasy.png")
```

This is basically a **free image generator model using Hugging Face** that you can plug into Flask, FastAPI, Streamlit, etc.

---

## 6️⃣ If You Tell Me Your Goal…

For example:

* “Use it in **Flask API**”
* “Use it in **Streamlit app**”
* “Use it in a **Jupyter Notebook** as a teaching demo”

…I can give you **full project code** (app.py + requirements.txt + sample prompts) tailored to that.

For now, you can copy-paste any of the above blocks and run them directly.
